In [324]:
import requests 
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import hashlib
import jupyter_util as util
from importlib import reload
reload(util)

<module 'jupyter_util' from '/home/sevni/Documents/thclab/aries/aries-acapy-plugin/controller_acapy/jupyter_util.py'>

## Enter Invite Url:

In [325]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiZmE4NWUzZWYtNzMzMC00NWVhLWE4YzYtNWZhZDBlYTdiMWJkIiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQxLmxvY2FsaG9zdCIsICJsYWJlbCI6ICJNYWluIChhZG1pbikiLCAicmVjaXBpZW50S2V5cyI6IFsiQ0U3NEhtclVNQ1hwUXk3d3RBbTZZdkNrUXA0QWhtbzV2Snk3c1pMSGpocFMiXX0=


In [326]:
agent2 = {}
endpoint = "http://localhost:8151/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent2['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent2['url'])

http://agent2.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiNjE1MzRjNmUtZGEzMS00MjY4LWJiMjAtYTM2MTNiZDZjYzdlIiwgImxhYmVsIjogIkNsaWVudCAoYWRtaW4pIiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQyLmxvY2FsaG9zdCIsICJyZWNpcGllbnRLZXlzIjogWyI4U1lkTEY2NTFMUjlYNWlBYmdnTFJ3cFA3VWlzMk1HSm9zaE1ES1VaYTV5RCJdfQ==


In [327]:
agent1 = util.processInviteUrl(agent1['url'])
agent2 = util.processInviteUrl(agent2['url'])
print("Decoded invitation:\n")
pp.pprint(agent2['invite'])

Decoded invitation:

{'@id': '61534c6e-da31-4268-bb20-a3613bd6cc7e',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Client (admin)',
 'recipientKeys': ['8SYdLF651LR9X5iAbggLRwpP7Uis2MGJoshMDKUZa5yD'],
 'serviceEndpoint': 'http://agent2.localhost'}


In [328]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 VdRxhV4RRxenpsp8RNJqTJuiFtbA9Eu5XeJh1WzXJDA7EqcDFFuJT91Y3ZswUqexhHc3utFQpgzVs1iQ1eZE9tZ

Public key: 
 85W41FpdcuJ4KNZvnGAmktLyUZihtDRqEWwwiED1PjNV


**Sending a connection request to acapy**

In [329]:
agent1['connection'] = util.connectWithAcapy(agent1, plugin)
agent2['connection'] = util.connectWithAcapy(agent2, plugin)
# print("Decoded acapy response: \n")
# pp.pprint(agent1['conn'])

**Unpacking connection data embedded in the response**

In [330]:
agent1['connection_details'] = util.decodeConnectionDetails(agent1['connection'])
agent2['connection_details'] = util.decodeConnectionDetails(agent2['connection'])

**Adding the connection to connection list**

In [331]:
agents = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": agent1['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
    {
        "label": agent2['invite']['label'],
        "DIDDoc": agent2['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
]

# pp.pprint(connections)

In [332]:
connection = util.connectAgents(agents[0], agents[1])
connection[1] = connection[1]['raw_repr']['connection_id']
connection[0] = connection[0]['connection_id']
print(connection[0], "\n", connection[1])

46ff09c7-afaa-4dfc-8cb4-a651d96ac8d5 
 ea676198-ca20-43d4-b96b-7b445d95a7a1


### FEATURE DISCOVERY

In [333]:
# pp.pprint(util.getFeatureDiscovery(agents[0]))

In [334]:
add = requests.post('http://localhost:8151/verifiable-services/add', json={
  "label": "memxsassdsssdfdeds",
  "service_schema": {
    "oca_schema_namespace": "string",
    "oca_schema_dri": "string"
  },
  "consent_schema": {
    "oca_schema_namespace": "string",
    "oca_schema_dri": "string",
    "data_url": "string"
  }
})
print(add.text)
add = json.loads(add.text)

{"updated_at": "2020-08-25 10:28:26.067345Z", "label": "memxsassdsssdfdeds", "consent_schema": {"data_url": "string", "oca_schema_namespace": "string", "oca_schema_dri": "string"}, "created_at": "2020-08-25 10:28:26.067345Z", "service_schema": {"oca_schema_namespace": "string", "oca_schema_dri": "string"}}


In [335]:
# did = requests.post('http://localhost:8151/wallet/did/create')
# did = json.loads(did.text)
# print(did)

In [336]:
# create_did = requests.post('http://localhost:8151/wallet/did/create')
# create_did = json.loads(create_did.text)
# print(create_did)

In [337]:
# register_did = requests.post('http://localhost:8151/ledger/register-nym', params={
#     "verkey": create_did['result']['verkey'],
#     "did": create_did['result']['did'],
# })
# register_did = json.loads(register_did.text)
# print(register_did)

In [338]:
# did = requests.post('http://localhost:8151/wallet/did/public', params={
#     'did': register_did['result']['did']
# })
# did = json.loads(did.text)
# print(did)

In [339]:
## DID

In [340]:
newDid1 = util.createDid(agents[1])
util.registerDidOnLedger(agents[1], newDid1['result']['did'], newDid1['result']['verkey'])

KeyboardInterrupt: 

In [341]:
newDid0 = util.createDid(agents[0])
util.registerDidOnLedger(agents[0], newDid0['result']['did'], newDid0['result']['verkey'])

KeyboardInterrupt: 

In [342]:
util.setPublicDid(agents[1], newDid1['result']['did'])
util.setPublicDid(agents[0], newDid0['result']['did'])

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/did',
 '@id': 'e62688ce-8b0b-4a1c-b1a5-c5ce8c3f019a',
 '~thread': {'thid': 'e3678c6b-90ad-4b20-9483-88e927c0141b'},
 'result': {'metadata': {'public': True},
  'did': 'LaeJo3amdkvMgXb5gqG6a3',
  'verkey': 'BfxvRDkn2jd9q8fqEeLzEDuGHrytCS4yVt3K1WxLhNun'}}

In [347]:
service_list = requests.get('http://localhost:8150/verifiable-services/request-service-list/' + connection[0])
print(service_list)
service_list = json.loads(service_list.text)
pp.pprint(service_list)

<Response [200]>
'SUCCESS: request sent, expect a webhook notification'


In [348]:
service_list = requests.get('http://localhost:8150/verifiable-services/DEBUGrequest/' + connection[0])
service_list = json.loads(service_list.text)
pp.pprint(service_list)

{'connection_id': '46ff09c7-afaa-4dfc-8cb4-a651d96ac8d5',
 'created_at': '2020-08-25 10:28:45.466420Z',
 'services': [{'consent_schema': {'data_url': 'string',
                                  'oca_schema_dri': 'string',
                                  'oca_schema_namespace': 'string'},
               'label': 'memxsassdsssdfdeds',
               'service_id': '0dc46016-c2df-4c54-962f-434187644722',
               'service_schema': {'oca_schema_dri': 'string',
                                  'oca_schema_namespace': 'string'}},
              {'consent_schema': {'data_url': 'string',
                                  'oca_schema_dri': 'string',
                                  'oca_schema_namespace': 'string'},
               'label': 'memxsassdsssdfdeds',
               'service_id': '867f5ba9-b588-42db-9e28-9ef2007620b3',
               'service_schema': {'oca_schema_dri': 'string',
                                  'oca_schema_namespace': 'string'}},
              {'consent_sche

In [345]:
apply = requests.post('http://localhost:8150/verifiable-services/apply', json={
  "connection_id": connection[0],
  "service_id": service_list['services'][0]['service_id'],
  "payload": "abc",
  "service": service_list['services'][0],
})
apply = json.loads(apply.text)
print(apply)

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/verifiable-services/1.0/application', '@id': '76314309-95b4-49b2-a478-d0bfb834be90', 'credential_definition_id': 'LaeJo3amdkvMgXb5gqG6a3:3:CL:595:consent_schema', 'data_dri': '23607c8e8b252f0c18468243ca3a50eaa70a3ef6704cd5220c932a655063b4f2', 'exchange_id': 'a9a9b0ee-b8ef-4442-a787-7f237fccfd88', 'service_id': '0dc46016-c2df-4c54-962f-434187644722'}


In [225]:
status = requests.get('http://localhost:8150/verifiable-services/apply-status', json={
  "connection_id": connection[0],
    "exchange_id": apply['exchange_id'],
})
status = json.loads(status.text)
pp.pprint(status)

[{'author': 'self',
  'connection_id': '4cc0a2d8-4df1-41a6-a5e4-345e05a425ab',
  'created_at': '2020-08-25 09:30:18.525070Z',
  'exchange_id': '6bd7aad2-6134-4067-b64d-418131f3fae4',
  'label': 'memxsassdsssdfdeds',
  'service_id': '681d61fa-b6d5-4b7a-9ac5-a45dfa36fd11',
  'state': 'pending',
  'updated_at': '2020-08-25 09:30:18.600280Z'}]


In [228]:
issue_self = requests.post('http://localhost:8151/verifiable-services/get-issue-self', json={
    "exchange_id": apply['exchange_id'],
})
issue_self = json.loads(issue_self.text)
pp.pprint(issue_self)

[{'author': 'other',
  'connection_id': 'bdf2e3c9-5dc1-4eb1-8400-69e058b800d0',
  'consent_schema': '{"oca_schema_namespace": "string", "oca_schema_dri": '
                    '"string", "data_url": "string"}',
  'created_at': '2020-08-25 09:30:18.567927Z',
  'exchange_id': '6bd7aad2-6134-4067-b64d-418131f3fae4',
  'issue_id': '1576894a38c80f5001377976d0f2a3668cc416aa76786870919469e8e8e18523',
  'label': 'memxsassdsssdfdeds',
  'payload': 'abc',
  'service_id': '681d61fa-b6d5-4b7a-9ac5-a45dfa36fd11',
  'service_schema': '{"oca_schema_namespace": "string", "oca_schema_dri": '
                    '"string"}',
  'state': 'accepted',
  'updated_at': '2020-08-25 09:30:32.048980Z'}]


In [227]:
issue_self = requests.post('http://localhost:8151/verifiable-services/process-application', json={
    "issue_id": issue_self[0]['issue_id'],
    "decision": "accept",
})
print(issue_self)
issue_self = json.loads(issue_self.text)
pp.pprint(issue_self)

<Response [200]>
{'credential_exchange_record': {'auto_issue': True,
                                'auto_offer': False,
                                'auto_remove': False,
                                'connection_id': 'bdf2e3c9-5dc1-4eb1-8400-69e058b800d0',
                                'created_at': '2020-08-25 09:30:32.040804Z',
                                'credential_definition_id': '7ktHcMmM5NK61P9g5Hio4P:3:CL:564:Services',
                                'credential_exchange_id': 'c27d7b52-650d-4b4f-bada-deb4bd82ce9b',
                                'credential_offer': {'cred_def_id': '7ktHcMmM5NK61P9g5Hio4P:3:CL:564:Services',
                                                     'key_correctness_proof': {'c': '110975624205282597727392057631075725644309954786316201575391002893922349344925',
                                                                               'xr_cap': [['oca_schema_dri',
                                                                     

In [36]:
payload = "some string yes dd"
payload_response = "response yay"
hash_id = hashlib.sha256(payload.encode("UTF-8")).hexdigest()
hash_id_response = hashlib.sha256(payload_response.encode("UTF-8")).hexdigest()
print(payload, hash_id)

some string yes dd b7c34035833ad394aaf721e431ae4a7f6359d66e1c69e8be9ec4c3e1d2e1ea1f


In [293]:
credentials = requests.get('http://localhost:8150/credentials')
credentials = json.loads(credentials.text) 
pp.pprint(credentials)

{'results': [{'attrs': {'consent_schema': '{"oca_schema_namespace": "string", '
                                          '"oca_schema_dri": "string", '
                                          '"data_url": "string"}',
                        'label': 'memxsassdsssdfdeds',
                        'service_schema': '{"oca_schema_namespace": "string", '
                                          '"oca_schema_dri": "string"}'},
              'cred_def_id': 'B6hy9V4FLjh6tQmknfEvTt:3:CL:273:Services',
              'cred_rev_id': None,
              'referent': 'e56cb0b5-f044-4176-b4d8-246c27ea0d00',
              'rev_reg_id': None,
              'schema_id': 'B6hy9V4FLjh6tQmknfEvTt:2:memxsassdsssdfdeds:1.0'},
             {'attrs': {'consent_schema': '{"oca_schema_namespace": "string", '
                                          '"oca_schema_dri": "string", '
                                          '"data_url": "string"}',
                        'label': 'memxsassdsssdfdeds',
         

I added returning hash id to add schema, I need to test if it works with both rejecting (not found schema) and accepting schema dound

### Add schema to storage

In [52]:
add = requests.post('http://localhost:8150/schema-storage/add', json={
  "payload": "test"
})
print(add.text)
add = json.loads(add.text)

{"hash_id": "9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08", "payload": "test"}


### SEND Initial Request

In [53]:
send = requests.post('http://localhost:8151/schema-exchange/request-schema', json={
  "hash_id": add['hash_id'],
  "connection_id": connection[1]
})

In [54]:
send = requests.post('http://localhost:8151/schema-exchange/request-schema', json={
  "hash_id": "ddd",
  "connection_id": connection[1]
})

In [55]:
print(send.text)

{"hash_id": "ddd", "connection_id": "a361d994-aaea-4f5c-9bf4-9644a774de2b", "record_id": "1e61b917-946b-4a56-8810-5b48081f79e6", "exchange_id": "c5f71769-b806-49fe-8861-15f49ffad9e3", "state": "pending"}


### GET Request By RECORD_ID

### GET Request List and filter by state and author

In [56]:
request_list = requests.post('http://localhost:8150/schema-storage/debug/request/list', json={})
request_list = json.loads(request_list.text)
print("Agent1:", )
pp.pprint(request_list)
request_list = requests.post('http://localhost:8151/schema-storage/debug/request/list', json={})
request_list = json.loads(request_list.text)
print("\nAgent2:")
pp.pprint(request_list)

Agent1:
[{'author': 'other',
  'connection_id': '844f6e7f-c13e-4ef8-b018-754fc5205445',
  'created_at': '2020-07-16 09:09:17.991960Z',
  'exchange_id': 'c5f71769-b806-49fe-8861-15f49ffad9e3',
  'payload': 'ddd',
  'state': 'failed to find the record',
  'updated_at': '2020-07-16 09:09:17.991960Z'},
 {'author': 'other',
  'connection_id': '844f6e7f-c13e-4ef8-b018-754fc5205445',
  'created_at': '2020-07-16 09:09:16.748526Z',
  'exchange_id': 'f51e4a06-2214-4b9e-a28a-d4c336258f66',
  'payload': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'state': 'approved',
  'updated_at': '2020-07-16 09:09:16.748526Z'}]

Agent2:
[{'author': 'self',
  'connection_id': 'a361d994-aaea-4f5c-9bf4-9644a774de2b',
  'created_at': '2020-07-16 09:09:17.950785Z',
  'exchange_id': 'c5f71769-b806-49fe-8861-15f49ffad9e3',
  'payload': 'ddd',
  'state': 'failed to find the record',
  'updated_at': '2020-07-16 09:09:18.029400Z'},
 {'author': 'self',
  'connection_id': 'a361d994-aaea-4f5c-9bf4-

### GET RECORD LIST

In [57]:
getRequests = requests.post('http://localhost:8150/schema-storage/list')
record_list = json.loads(getRequests.text)
print("Agent1: ")
pp.pprint(record_list)

getRequests = requests.post('http://localhost:8151/schema-storage/list')
record_list = json.loads(getRequests.text)
print("Agent2: ")
pp.pprint(record_list)

Agent1: 
[{'author': 'self',
  'created_at': '2020-07-16 09:09:15.407083Z',
  'hash_id': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'payload': 'test',
  'updated_at': '2020-07-16 09:09:15.407083Z'}]
Agent2: 
[{'author': 'a361d994-aaea-4f5c-9bf4-9644a774de2b',
  'created_at': '2020-07-16 09:09:16.808608Z',
  'hash_id': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'payload': 'test',
  'updated_at': '2020-07-16 09:09:16.808608Z'}]


In [ ]:

# message = util.buildMessage(
#         'https://didcomm.org/issue-credential/1.0/propose-credential'
#     )
# testing = util.sendMessage(message, agents[0])
# pp.pprint(testing)

In [ ]:
# pp.pprint(util.sendSchema(agents[0], "schesssdddvd", "1.0", ["memesssaaaaaaa"]))

In [ ]:
# result = util.schemaGetList(agents[0])
# last_schema_id = result['results'][-1]['schema_id']
# print(result)

In [ ]:
# util.schemaGet(agents[0], last_schema_id)